In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, DepthwiseConv2D


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchsummary import summary
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from time import time

Question 1 -
Implement 3 different CNN architectures with a comparison table for the MNSIT
dataset using the Tensorflow library.
Note -
1. The model parameters for each architecture should not be more than 8000
parameters
2. Code comments should be given for proper code understanding.
3. The minimum accuracy for each accuracy should be at least 96%

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
# Normalise the pixel values to range 0-1 and reshape
x_train = x_train.reshape((x_train.shape[0], 28,28,1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0],28,28,1)).astype('float32') / 255

In [5]:
#one hot encoding the lables
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Architecture 1: Small CNN with 6,810 parameters
model1 = models.Sequential([
    layers.Conv2D(4, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Architecture 2: Depthwise CNN with 6,650  parameters
model2 = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.DepthwiseConv2D((3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Architecture 3: Strided CNN with 5,930 parameters
model3 = models.Sequential([
    layers.Conv2D(20, (3, 3), strides=(2,2), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])


# Function to compile, train, and evaluate models
def train_and_evaluate(model, x_train, y_train, x_test, y_test, optimizer='adam', epochs=5, batch_size=64):
    print("optimizer", optimizer, "epochs", epochs, "batch_size", batch_size)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return test_acc, model.count_params()

# Train and evaluate all models
results = {}
models_list = [model1, model2, model3]
model_names = ['Model 1', 'Model 2', 'Model 3']
optimizers = ['adam', 'sgd', 'rmsprop']
epochs = [15, 10, 7]

for model, name, optimizer, epoch in zip(models_list, model_names, optimizers, epochs):
    acc, params = train_and_evaluate(model, x_train, y_train, x_test, y_test, optimizer, epoch)
    results[name] = {'Accuracy': acc, 'Parameters': params, 'Optimizer':optimizer}

# Print results in a comparison table
print(f"{'Model':<10} {'Accuracy':<10} {'Parameters':<10} {'Optimizer':<10}")
for model, metrics in results.items():
    print(f"{model:<10} {metrics['Accuracy']:<10.4f} {metrics['Parameters']:<10} {metrics['Optimizer']:10}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


optimizer adam epochs 15 batch_size 64
optimizer sgd epochs 10 batch_size 64
optimizer rmsprop epochs 7 batch_size 64
Model      Accuracy   Parameters Optimizer 
Model 1    0.9699     6810       adam      
Model 2    0.9800     6650       sgd       
Model 3    0.9735     7410       rmsprop   


Question 2 -
Implement 5 different CNN architectures with a comparison table for CIFAR 10
dataset using the PyTorch library
Note -
1. The model parameters for each architecture should not be more than 10000
parameters
2. Code comments should be given for proper code understanding

In [7]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using Device {device}")
if torch.cuda.is_available():
  print(f"cuda name is {torch.cuda.get_device_name(0)}")
# Hyperparameters
batch_size = 32
learning_rate = 0.001

# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

print(train_dataset.data.shape[1:])
print(train_loader.dataset.data.shape[1:])

using Device cuda
cuda name is Tesla T4


100%|██████████| 170M/170M [00:03<00:00, 49.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
(32, 32, 3)
(32, 32, 3)


In [8]:
def count_trainable_params(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [17]:
class CNN1(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=8, out_channels=13, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(13 * 8 * 8, 10)
    )
    self.to(device)


  def forward(self, x):
    x = self.conv1(x)
    return x


class CNN2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1), # (input_width - Filter_size + 2(padding)) / Stripe + 1 = (32 - 3 + 2(0)) / 2 + 1
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # = (16-2)/1 +1
        nn.Conv2d(in_channels=16, out_channels=31, kernel_size=3, stride=2, padding=1), # (16 - 3)  / 1 +1
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(31 * 4 * 4, 10)
    )
    self.to(device)
  def forward(self, x):
    x = self.conv1(x)
    return x


class CNN3(nn.Module):
  def __init__(self):
    super(CNN3, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=14, kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(14 * 8 * 8, 10)
    )
    self.to(device)

  def forward(self, x):
      x = self.conv1(x)
      return x

class CNN4(nn.Module):
  def __init__(self):
    super(CNN4, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(16 * 7 * 7, 10)
    )
    self.to(device)

  def forward(self, x):
    x = self.conv(x)
    return x

class CNN5(nn.Module):
  def __init__(self):
    super(CNN5, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=6, out_channels=14, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(14 * 8 * 8, 10)
    )
    self.to(device)

  def forward(self, x):
    x = self.conv(x)
    return x


In [18]:
def train_and_evaluate(model, optimizer_name='adam', epochs=5, lr=0.001):
  model_name = model.__class__.__name__
  criterion = nn.CrossEntropyLoss()
  if optimizer_name.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'sgd':
      optimizer = optim.SGD(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'rms':
      optimizer = optim.RMSprop(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'adadelta':
      optimizer = optim.Adadelta(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'adagrad':
      optimizer = optim.Adagrad(model.parameters(), lr=lr)
  else:
      raise ValueError("Invalid optimizer name. Choose from 'adam', 'sgd', 'rms', 'adadelta' or 'adagrad'.")
  start_time = time()
  for epoch in range(epochs):
      for i, (images, labels) in enumerate(train_loader):
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (i + 1) % 1500 == 0:
              print(f'{model_name} Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

  end_time = time()
  print(f"Model name: {model_name}, Training time: {(end_time - start_time):.2f}s")

  # Evaluation
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      accuracy = 100 * correct / total
      print(f'Accuracy of the model {model_name} on the test images: {accuracy:.2f}%')
      return accuracy, count_trainable_params(model), end_time - start_time


In [19]:
# Train and evaluate all models
results = {}
models_list = [CNN1(), CNN2(), CNN3(), CNN4(), CNN5()]
model_names = ['Model_1','Model_2', 'Model_3', 'Model_4', 'Model_5']
optimizers = ['rms', 'adam', 'sgd', 'adam', 'adam']
epochs = [5, 5, 5, 5, 5]

for model, name, optimizer, epoch in zip(models_list, model_names, optimizers, epochs):
    acc, params, training_time = train_and_evaluate(model, optimizer, epoch)
    results[name] = {'Accuracy': acc, 'Parameters': params, 'Optimizer':optimizer, 'training_time': training_time}

# Print results in a comparison table
print(f"{'Model':<10} {'Accuracy':<10} {'Parameters':<10} {'Optimizer':<10} {'Training time':<10}")
for model, metrics in results.items():
    print(f"{model:<10} {metrics['Accuracy']:<10.3f} {metrics['Parameters']:<10} {metrics['Optimizer']:10} {metrics['training_time']:10.2f}")

CNN1 Epoch [1/5], Step [1500/1563], Loss: 1.6611
CNN1 Epoch [2/5], Step [1500/1563], Loss: 1.4203
CNN1 Epoch [3/5], Step [1500/1563], Loss: 1.0885
CNN1 Epoch [4/5], Step [1500/1563], Loss: 0.8662
CNN1 Epoch [5/5], Step [1500/1563], Loss: 1.0304
Model name: CNN1, Training time: 76.94s
Accuracy of the model CNN1 on the test images: 60.49%
CNN2 Epoch [1/5], Step [1500/1563], Loss: 1.2516
CNN2 Epoch [2/5], Step [1500/1563], Loss: 1.1318
CNN2 Epoch [3/5], Step [1500/1563], Loss: 0.9055
CNN2 Epoch [4/5], Step [1500/1563], Loss: 0.8212
CNN2 Epoch [5/5], Step [1500/1563], Loss: 1.0036
Model name: CNN2, Training time: 77.24s
Accuracy of the model CNN2 on the test images: 64.43%
CNN3 Epoch [1/5], Step [1500/1563], Loss: 1.9959
CNN3 Epoch [2/5], Step [1500/1563], Loss: 1.8903
CNN3 Epoch [3/5], Step [1500/1563], Loss: 1.7466
CNN3 Epoch [4/5], Step [1500/1563], Loss: 1.5043
CNN3 Epoch [5/5], Step [1500/1563], Loss: 1.7190
Model name: CNN3, Training time: 72.44s
Accuracy of the model CNN3 on the tes

Question 3 -
Train a Pure CNN with less than 10000 trainable parameters using the MNIST
Dataset having minimum validation accuracy of 99.40%
Note -
1. Code comments should be given for proper code understanding.
2. Implement in both PyTorch and Tensorflow respectively

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalise the pixel values to range 0-1 and reshape
x_train = x_train.reshape((x_train.shape[0], 28,28,1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0],28,28,1)).astype('float32') / 255

#one hot encoding the lables
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [14]:
model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.Conv2D(18, (3, 3), activation='relu', strides=(2,2)),
        # layers.MaxPooling2D((2, 2)),
        layers.Conv2D(18, (3, 3), activation='relu', strides=(2,2)),
        layers.Conv2D(20, (3, 3), activation='relu'),
        layers.GlobalAveragePooling2D(),
        # layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ])

print(model.count_params())

9174


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 26, 26, 16)          │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 12, 12, 18)          │           2,610 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 5, 5, 18)            │           2,934 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 3, 3, 20)            │           3,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 20)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             210 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,174 (35.84 KB)

 Trainable params: 9,174 (35.84 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
x_train = x_train[..., tf.newaxis]  # Add channel dimension
x_test = x_test[..., tf.newaxis]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Split into training and validation sets
x_train, x_val = x_train[:50000], x_train[50000:]
y_train, y_val = y_train[:50000], y_train[50000:]

# Define the CNN model
def create_model():
    model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.Conv2D(18, (3, 3), activation='relu', strides=(2,2)),
        # layers.MaxPooling2D((2, 2)),
        layers.Conv2D(18, (3, 3), activation='relu', strides=(2,2)),
        layers.Conv2D(20, (3, 3), activation='relu'),
        layers.GlobalAveragePooling2D(),
        # layers.Flatten(),
        layers.Dense(10, activation='softmax')
    ])
    return model

model = create_model()
model.summary()

# Check the parameter count
assert model.count_params() < 10000, f"Model has {model.count_params()} parameters, exceeds limit."

# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training with early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]

history = model.fit(x_train, y_train, epochs=50, batch_size=64,
                    validation_data=(x_val, y_val),
                    callbacks=callbacks)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 26, 26, 16)          │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 12, 12, 18)          │           2,610 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 5, 5, 18)            │           2,934 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 3, 3, 20)            │           3,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 20)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │             210 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,174 (35.84 KB)

 Trainable params: 9,174 (35.84 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.6151 - loss: 1.1281 - val_accuracy: 0.9298 - val_loss: 0.2445
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9281 - loss: 0.2433 - val_accuracy: 0.9538 - val_loss: 0.1539
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9456 - loss: 0.1825 - val_accuracy: 0.9619 - val_loss: 0.1349
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9570 - loss: 0.1377 - val_accuracy: 0.9593 - val_loss: 0.1336
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9629 - loss: 0.1234 - val_accuracy: 0.9598 - val_loss: 0.1259
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9670 - loss: 0.1073 - val_accuracy: 0.9724 - val_loss: 0.0913
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9711 - loss: 0.0953 - val_accuracy: 0.9769 - val_loss: 0.0836
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9743 - loss: 0.0820 - val_accuracy: 0.

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


print(train_dataset.data.shape[1:])
print(train_loader.dataset.data.shape[1:])

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.0MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 485kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.44MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.00MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

torch.Size([28, 28])
torch.Size([28, 28])


In [26]:
class PytorchModel(nn.Module):
  def __init__(self):
    super(PytorchModel, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),  # Input: 28x28 -> Output: 28x28 # Added padding=1
        nn.ReLU(),
        nn.Conv2d(8, 12, kernel_size=3, stride=2, padding=1),  # Input: 28x28 -> Output: 14x14 # Added padding=1
        nn.ReLU(),
        nn.Conv2d(12, 16, kernel_size=3, stride=2, padding=1),  # Input: 14x14 -> Output: 7x7 # Added padding=1
        nn.ReLU(),
        nn.Conv2d(16, 20, kernel_size=3, stride=2, padding=1),  # Input: 7x7 -> Output: 4x4 # Added padding=1, adjusted stride to 2
        nn.ReLU(),
        nn.Conv2d(20, 22, kernel_size=3, stride=1, padding=1),  # Input: 4x4 -> Output: 4x4  # Added padding=1
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling: Output: 20x1x1
        nn.Flatten(),  # Flatten the output for Dense layer
        nn.Linear(22, 10),  # Fully connected layer with 10 outputs
    )
    self.to(device)

  def forward(self, x):
    x = self.conv(x)
    return x

# Check parameter count
model = PytorchModel()
param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {param_count}")

# Ensure parameters are under 10,000
assert param_count < 10000, "Model exceeds parameter limit!"

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 30
start_time = time()
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 450 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Evaluation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy on the test images: {accuracy:.2f}%')

end_time = time()
print(f"Training time: {(end_time - start_time):.2f}s")

Trainable parameters: 9812
Epoch [1/30], Step [450/938], Loss: 0.3604
Epoch [1/30], Step [900/938], Loss: 0.1633
Accuracy on the test images: 92.62%
Epoch [2/30], Step [450/938], Loss: 0.0773
Epoch [2/30], Step [900/938], Loss: 0.1496
Accuracy on the test images: 95.21%
Epoch [3/30], Step [450/938], Loss: 0.2196
Epoch [3/30], Step [900/938], Loss: 0.0953
Accuracy on the test images: 96.29%
Epoch [4/30], Step [450/938], Loss: 0.0309
Epoch [4/30], Step [900/938], Loss: 0.2427
Accuracy on the test images: 96.71%
Epoch [5/30], Step [450/938], Loss: 0.0773
Epoch [5/30], Step [900/938], Loss: 0.0249
Accuracy on the test images: 96.94%
Epoch [6/30], Step [450/938], Loss: 0.0456
Epoch [6/30], Step [900/938], Loss: 0.0599
Accuracy on the test images: 97.07%
Epoch [7/30], Step [450/938], Loss: 0.0251
Epoch [7/30], Step [900/938], Loss: 0.0940
Accuracy on the test images: 97.89%
Epoch [8/30], Step [450/938], Loss: 0.0417
Epoch [8/30], Step [900/938], Loss: 0.1051
Accuracy on the test images: 97.5